In [1]:
import openai
import json
import os
from sympy import symbols, Symbol

openai.api_key = open("/home/mcwave/code/automath/openai_api_key.txt").readline().strip()
#print(openai.api_key)

def convertToSympyWithGPT(prompt):
    messages = [{"role": "user", "content": "Convert to Sympy with a symbol and an expression: $" + prompt + '$'}]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        timeout=10
    )
    response_message = response["choices"][0]["message"]
    return response_message


def convertToSympy(prompt):
    response = convertToSympyWithGPT(prompt)["content"]
    #print(response)
    lcls = locals()
    if 't' in lcls:
        del t
    if 'expression' in lcls:
        del expression
    if 'expr' in lcls:
        del expr
    #
    lines = response.split('\n')
    #print(lines)
    for line in lines:
        exec(line, globals(), lcls)
    #
    output = None
    if "expression" in lcls:
        output = lcls["expression"]
    elif "expr" in lcls:
        output = lcls["expr"]
    #
    return lcls["t"], output

def convertToSympyWithRetry(prompt, retry=3):
    success = False
    for i in range(retry):
        try:
            sbl, expr = convertToSympy(prompt)
            success = True
        except:
            continue
        if sbl is None or expr is None:
            continue
        break
    if success:
        return sbl, expr
    else:
        return None, None

convertToSympyWithRetry('tan (7-7 t)') #'-\frac{3150 t^2-10920 t+9437}{3 \sqrt{3}}') #'-\\frac{17 t^2}{8}+\\frac{255 t}{2}-1906')

(t, -tan(7*t - 7))

In [2]:
def extract_equations(s):
    in_equation = False
    cur_equation = ""
    equations = []
    for i in range(len(s)):
        if s[i]=='$':
            in_equation = not in_equation
            if not in_equation:
                if len(cur_equation) > 0:
                    if ',' in cur_equation:
                        cur_equations = cur_equation.split(',')
                        for e in cur_equations:
                            equations.append(e.strip())
                    else:
                        equations.append(cur_equation)
                    cur_equation = ""
        elif in_equation:
            cur_equation += s[i]
    return equations
                
def parse_parametric_equations(path):
    fin = open(path, 'r')
    lines = fin.readlines()
    fin.close()
    line = ' '.join(lines)
    eqts = extract_equations(line)
    print(eqts)
    f_t, g_t, g_f_t = None, None, None
    for eqt in eqts:
        if len(eqt) <= 10:
            continue
        if eqt.startswith('x(t)'):
            f_t = eqt[eqt.find('=') + 1:]
        elif eqt.startswith('y(t)'):
            g_f_t = eqt[eqt.find('=') + 1:]
        elif eqt.startswith('y=') or eqt.startswith('y ='):
            g_t = eqt[eqt.find('=') + 1:]
            g_t = g_t.replace('x', 't')
    return f_t, g_t, g_f_t

def parse_function_evaluation(path):
    fin = open(path, 'r')
    lines = fin.readlines()
    fin.close()
    line = ' '.join(lines)
    eqts = extract_equations(line)
    eqt = eqts[0]
    if eqt.startswith('f(x)'):
        f_t = eqt[eqt.find('=') + 1:].replace('x', 't')
        return f_t
    else:
        return None


path = '/home/mcwave/code/automath/calculus/datasets/function_evaluation/123.txt'
f_x = parse_function_evaluation(path)
print(f_x)

-\tan (8 t)


In [3]:
# import os
# import pickle

# directory = '/home/mcwave/code/automath/calculus/datasets/parametric_equations/'
# files = os.listdir(directory)

# json_path = '/home/mcwave/code/automath/calculus/datasets/parametric_equations.json'
# pickle_path = '/home/mcwave/code/automath/calculus/datasets/parametric_equations.pkl'

# processed_files = set()
# fin = open(json_path, 'r')
# lines = fin.readlines()
# for line in lines:
#     parsed = json.loads(line)
#     processed_files.add(parsed["filename"])

# fin.close()
# print(len(processed_files), "already processed")
    
# fout = open(json_path, 'a')
# pickleout = open(pickle_path, 'ab')

# for file in files:
#     if file in processed_files:
#         print(file, "processed")
#         continue
#     if file.endswith(".txt"):
#         print(file)
#         path = os.path.join(directory, file)
#         f_t, g_t, g_f_t = parse_parametric_equations(path)
#         parsed = {'f_t':f_t, 'g_t':g_t, 'g_f_t':g_f_t}
#         output = {"filename":file}
#         outputs = {"filename":file}
#         for k, v in parsed.items():
#             sbl, expr = convertToSympyWithRetry(v)
#             if sbl is None or expr is None:
#                 continue
#             output[k] = (sbl, expr)
#             outputs[k] = (str(sbl),str(expr))
#         if len(output) > 1:
#             pickle.dump(output, pickleout)
#             pickleout.flush()
#             fout.write(json.dumps(outputs) + '\n')
#             fout.flush()

# fout.close()
# pickleout.close()

In [ ]:
import os
import pickle

directory = '/home/mcwave/code/automath/calculus/datasets/function_evaluation/'
files = os.listdir(directory)

json_path = '/home/mcwave/code/automath/calculus/datasets/function_evaluation.json'
pickle_path = '/home/mcwave/code/automath/calculus/datasets/function_evaluation.pkl'

processed_files = set()
fin = open(json_path, 'r')
lines = fin.readlines()
for line in lines:
    parsed = json.loads(line)
    processed_files.add(parsed["filename"])

fin.close()
print(len(processed_files), "already processed")
    
fout = open(json_path, 'a')
pickleout = open(pickle_path, 'ab')

for file in files:
    if file in processed_files:
        print(file, "processed")
        continue
    if file.endswith(".txt"):
        print(file)
        path = os.path.join(directory, file)
        f_t = parse_function_evaluation(path)
        print(f_t)
        parsed = {'f_t':f_t}
        output = {"filename":file}
        outputs = {"filename":file}
        for k, v in parsed.items():
            sbl, expr = convertToSympyWithRetry(v)
            print(sbl, expr)
            if sbl is None or expr is None:
                continue
            output[k] = (sbl, expr)
            outputs[k] = (str(sbl),str(expr))
        if len(output) > 1:
            pickle.dump(output, pickleout)
            pickleout.flush()
            fout.write(json.dumps(outputs) + '\n')
            fout.flush()

fout.close()
pickleout.close()

24 already processed
11508.txt processed
7233.txt processed
23916.txt processed
4800.txt processed
29016.txt processed
26545.txt processed
25547.txt processed
21753.txt processed
8423.txt processed
25394.txt processed
43598.txt processed
18293.txt processed
3369.txt processed
23846.txt processed
30686.txt processed
15884.txt processed
43930.txt processed
45690.txt processed
13452.txt processed
23739.txt processed
47013.txt processed
14360.txt processed
22020.txt processed
29318.txt processed
29226.txt
-\sin \left(\frac{9 t}{2}+\frac{5}{2}\right)-\tan \left(\frac{9 t}{2}+8\right)
t -sin(9*t/2 + 2.5) - tan(9*t/2 + 8)
5889.txt
\sqrt[3]{3 t+4}


/home/mcwave/anaconda3/envs/symbolic/lib/python3.7/site-packages/ipykernel_launcher.py:1: SymPyDeprecationWarning: 

Eq(expr) with a single argument with the right-hand side
defaulting to 0 is deprecated. Use Eq(expr, 0) instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-eq-expr
for details.

This has been deprecated since SymPy version 1.5. It
will be removed in a future version of SymPy.

  """Entry point for launching an IPython kernel.


t Eq((3*t + 4)**(1/3), 0)
18603.txt
\tan \left(2-2 t^2\right)
t -tan(2*t**2 - 2)
29527.txt
\tan ^{-1}\left(\frac{9 t}{2}+4\right)
t atan(9*t/2 + 4)
48835.txt
\sqrt{5 t-\frac{1}{2}} \tan ^{-1}\left(\frac{11}{2}-\frac{5 t}{2}\right)
t -sqrt(5)*sqrt(t - 0.1)*atan(5*t/2 - 5.5)
22324.txt
e^{\cos \left(2 t^3+7\right)}
t exp(cos(2*t**3 + 7))
22970.txt
\sin \left(\sqrt[3]{6 t^2+5}\right)
sin((6*t**2 + 5)**(1/3))
t sin((6*t**2 + 5)**(1/3))
44459.txt
\sqrt[3]{7} \sqrt[3]{t} \tan ^{-1}(8 t+4)
t 1.91293118277239*t**0.333333333333333*atan(8*t + 4)
34711.txt
\sqrt[3]{3 t^2-4} \cot (4 t+5)
t sqrt(3*t**2 - 4)*cot(4*t + 5)
29094.txt
\cos (3 t+6)
t cos(3*t + 6)
30092.txt
\sqrt{-2 t-4}+\tan (3 t)
t sqrt(-2*t - 4) + tan(3*t)
36494.txt
\cos (3 t)+\cos \left(\tan \left(\frac{15}{2}-\frac{5 t}{2}\right)\right)
cos(3*t) + cos(tan(5*t/2 - 7.5))
t cos(3*t) + cos(tan(5*t/2 - 7.5))
13291.txt
\log (6-4 t) \cot (4 t+5)
t log(6 - 4*t)*cot(4*t + 5)
1005.txt
\cosh ^{-1}(2)-e^{8-5 t}
t -exp(8 - 5*t) + 1/cosh(2)
37251.t

t -sin(6*t + 6.33333333333333)
30520.txt
\frac{1}{\left(\frac{13}{2}-\frac{5 t}{2}\right)^2}+\cos \left(\frac{7 t}{2}+4\right)
cos(7*t/2 + 4) + 0.0236686390532544/(1 - 0.384615384615385*t)**2
t cos(7*t/2 + 4) + 0.0236686390532544/(1 - 0.384615384615385*t)**2
47388.txt
\sqrt[3]{3 t+6} \csc (t+5)
t (3*t + 6)**(1/3)*csc(t + 5)
9353.txt
-\frac{1}{4}
-0.250000000000000
-0.250000000000000
None None
7872.txt
\sqrt[3]{t+3}-\frac{1}{9 t}
-t/9 + (t + 3)**(1/3)
t -t/9 + (t + 3)**(1/3)
13415.txt
\sin (5-2 t)
t -sin(2*t - 5)
19893.txt
\frac{\sqrt{8 t^2-5}}{\left(7-4 t^5\right)^5}
t sqrt(8*t**2 - 5)/(7 - 4*t**5)**5
84.txt
-\sin \left(8 t^4+4\right)
t -sin(8*t**4 + 4)
692.txt
e^{-6 t^2-2}
exp(-6*t**2 - 2)
t exp(-6*t**2 - 2)
20526.txt
\frac{\cos (1-t)}{\log \left(8 t^4+2\right)}
t cos(t - 1)/log(8*t**4 + 2)
24299.txt
\cos ^{-1}\left(\sin \left(5 t^3+8\right)\right)
t asin(sin(5*t**3 + 8))
6516.txt
\tan (\cos (4 t+3))
t tan(cos(4*t + 3))
25752.txt
\cos \left(\frac{5 t}{2}+9\right)
t cos(5*t/2 + 9)
4992

exp(t + 4)
t exp(t + 4)
8712.txt
\tan (t+8) \tan (4 t+2)
tan(t + 8)*tan(4*t + 2)
t tan(t + 8)*tan(4*t + 2)
30669.txt
\cos (8 t+7)
cos(8*t + 7)
t cos(8*t + 7)
49369.txt
\frac{e^{9-5 t}}{2-\frac{7 t}{2}}
t exp(9 - 5*t)/(2 - 7*t/2)
38364.txt
\cos (3)
None None
9220.txt
\sin \left(\frac{3}{2}-5 t\right) \cos \left(2 t+\frac{13}{2}\right)
t -sin(5*t - 1.5)*cos(2*t + 6.5)
38538.txt
\sqrt[9]{4 t^5+2}
t (4*t**5 + 2)**(1/9)
17806.txt
-\tan \left(\frac{33 t^3}{5}+\frac{24}{5}\right)
t -tan(33*t**3/5 + 4.8)
47628.txt
-5 t+\sqrt[3]{e^{-9 t-7}}-9
t -5*t + exp(-9*t - 7)**(1/3) - 9
9318.txt
\sin \left(4-\frac{17 t}{2}\right)-\sin \left(4-5 t^5\right)
t -sin(17*t/2 - 4) + sin(5*t**5 - 4)
29930.txt
\sqrt[3]{\frac{5 t^2}{2}+6}+\tan (3-9 t)
t sqrt(5*t**2/2 + 6) - tan(9*t - 3)
31814.txt
\sqrt[3]{\frac{44}{5}-7 t}
t (44/5 - 7*t)**(1/3)
48359.txt
\tan (2 t+9) \cosh ^{-1}\left(2-3 t^3\right)
t tan(2*t + 9)*acosh(2 - 3*t**3)
41114.txt
\sqrt[3]{4-4 t}
t (4 - 4*t)**(1/3)
16598.txt
\cos (\tan (5-6 t))
t cos(tan(

...

-tan(t - 8)
t -tan(t - 8)
33206.txt
-\tan \left(9-t^2\right)-\tan ^{-1}(2-5 t)
t tan(t**2 - 9) + atan(5*t - 2)
16017.txt
\sqrt{t^4+5}-(5-5 t)^4
t -(5 - 5*t)**4 + sqrt(t**4 + 5)
32589.txt
e^{7-7 t}
exp(7 - 7*t)
t exp(7 - 7*t)
40015.txt
\sqrt[3]{\cos (8-3 t)}
t cos(3*t - 8)**0.333333333333333
27798.txt
\sqrt[3]{6-6 t}
(6 - 6*t)**(1/3)
t (6 - 6*t)**(1/3)
38676.txt
\frac{1}{(2 t+6)^2}-e^{t+8}
t -exp(t + 8) + (2*t + 6)**(-2)
8172.txt
-\frac{\tan \left(1-\frac{t}{2}\right)}{\sqrt{7-9 t}}
tan(t/2 - 1)/sqrt(7 - 9*t)
t tan(t/2 - 1)/sqrt(7 - 9*t)
15244.txt
\frac{e^{7 t-7}}{\log \left(t^2+1\right)}
t exp(7*t - 7)/log(t**2 + 1)
28274.txt
8 t+1+\tan \left(\frac{1}{2}\right)
t 8*t + 1.54630248984379
26250.txt
\frac{1}{-\frac{32 t}{5}-\frac{24}{5}}
-5/(32*t + 24.0)
t 1/(-32*t/5 - 4.8)
46711.txt
\sqrt[3]{3-\frac{5 t}{2}}
t (3 - 5*t/2)**(1/3)
15145.txt
-\tan \left(\frac{32}{5}-\frac{31 t^5}{5}\right)
tan(31*t**5/5 - 6.4)
t tan(31*t**5/5 - 6.4)
11171.txt
e^{8 t-\frac{5}{2}}
t 0.0820849986238988*exp(8*t)

t sqrt(cos(4*t + 1)) - tan(8*t**2 + 9)
20790.txt
\cos \left(2-\frac{17 t^2}{2}\right)
t cos(17*t**2/2 - 2)
32416.txt
\sqrt[3]{6-3 t}
t (6 - 3*t)**(1/3)
10279.txt
-\frac{1}{\left(3 t^4-7\right)^2}-\sin (5-9 t)
t sin(9*t - 5) - 1/(3*t**4 - 7)**2
32180.txt
\tan (8-3 t)-2
t -tan(3*t - 8) - 2
3058.txt
-\sin (2-5 t)-2
t sin(5*t - 2) - 2
3445.txt
\tan \left(3-7 t^5\right) \csc (7 t+9)
t -tan(7*t**5 - 3)*csc(7*t + 9)
24417.txt
\sqrt[3]{-2 t-\frac{1}{2}}+\tan (2-t)
1.12246204830937*(-t - 0.25)**0.166666666666667 - tan(t - 2)
t 1.12246204830937*(-t - 0.25)**0.166666666666667 - tan(t - 2)
3190.txt
\sqrt[3]{6} \sqrt[3]{t}
6**(1/3)*t**(1/3)
t 6**(1/3)*t**(1/3)
24494.txt
e^{8-3 t}
t exp(8 - 3*t)
9265.txt
\cos (2-t)
t cos(t - 2)
29820.txt
\tan (3 t+3)
t tan(3*t + 3)
38680.txt
-\sin \left(\frac{14}{3}-\frac{4 t}{3}\right)
t sin(4*t/3 - 4.66666666666667)
49011.txt
e^{5 t-7}-\tan ^{-1}\left(8-7 t^4\right)
t exp(5*t - 7) + atan(7*t**4 - 8)
15793.txt
\sqrt[3]{-6 t-4}
t (-6*t - 4)**(1/3)
46016.txt
\cos \le

cos(4*t**2 - 3)*cos(5*t**2 + 3)
t cos(4*t**2 - 3)*cos(5*t**2 + 3)
41277.txt
\sqrt{5} \tan ^{-1}\left(1-7 t^2\right)
t -sqrt(5)*atan(7*t**2 - 1)
15115.txt
e^{-\frac{5 t}{2}-\frac{17}{2}} \cot \left(\frac{3 t}{2}+\frac{3}{2}\right)
0.000203468369010644*exp(-5*t/2)*cot(3*t/2 + 1.5)
t 0.000203468369010644*exp(-5*t/2)*cot(3*t/2 + 1.5)
25889.txt
\log \left(\frac{3 t}{2}+4\right)-\tan \left(\frac{13}{2}-\frac{11 t}{2}\right)
log(3*t/2 + 4) + tan(11*t/2 - 6.5)
t log(3*t/2 + 4) + tan(11*t/2 - 6.5)
16811.txt
\log (-\sin (8-3 t))-\tan (4 t+3)
log(sin(3*t - 8)) - tan(4*t + 3)
t log(sin(3*t - 8)) - tan(4*t + 3)
17462.txt
\frac{e^{\frac{13 t}{2}-6}}{\frac{17}{2}-5 t}
t exp(13*t/2 - 6)/(8.5 - 5*t)
11381.txt
e^{-\frac{t}{2}-2}
t exp(-t/2 - 2)
20065.txt
-\tan \left(\frac{11 t}{3}+\frac{10}{3}\right)
t -tan(11*t/3 + 3.33333333333333)
38690.txt
\sqrt{-\frac{9 t}{2}-8}
sqrt(-9*t/2 - 8)
t sqrt(-9*t/2 - 8)
23124.txt
-3 t-7
-3*t - 7
t -3*t - 7
3017.txt
-\frac{\sin (3 t+2)}{\sqrt{(8-7 t)^2+1}}
t -sin(3*t + 2)

(7*t**3 - 9)**(1/3)
t (7*t**3 - 9)**(1/3)
12118.txt
\cos \left(\frac{15 t}{2}+5\right)-\sin \left(\frac{5 t^2}{2}+\frac{9}{2}\right)
t -sin(5*t**2/2 + 4.5) + cos(15*t/2 + 5)
7549.txt
\cos ^{-1}(\cos (2-6 t))
t acos(cos(6*t - 2))
36318.txt
-\tan (4 t+8)
-tan(4*t + 8)
t -tan(4*t + 8)
26474.txt
\cos (8-8 t)
t cos(8*t - 8)
8945.txt
\cos \left(\sin \left(\frac{1}{3}-t\right)\right)
t cos(sin(t - 0.333333333333333))
45250.txt
\tan \left(3-2 t^2\right)+\cos (4 t+2)
t cos(4*t + 2) - tan(2*t**2 - 3)
11763.txt
\sqrt[3]{6 t-1}-\sqrt{5 t+5}
t -sqrt(5*t + 5) + (6*t - 1)**0.333333333333333
40050.txt
\cos \left(2 t^2+3\right)
t cos(2*t**2 + 3)
43087.txt
-\sin (7-2 t) \sin (5 t+4)
t sin(2*t - 7)*sin(5*t + 4)
33790.txt
\cos (8-t) \sec (6 t+3)
cos(t - 8)*sec(6*t + 3)
t cos(t - 8)*sec(6*t + 3)
46659.txt
\frac{1}{(-9 t-8)^4}
(9*t + 8)**(-4)
t (-9*t - 8)**(-4)
15243.txt
\sqrt{4 t+1}
t sqrt(4*t + 1)
16951.txt
e^{\cos (8 t+6)} \tett{sech}(8-5 t)
t exp(cos(8*t + 6))*sech(5*t - 8)
3760.txt
\sqrt{6 t-1}
t sqrt(

t 1/(sqrt(6*t + 1)*cos(13*t/2 - 5))
17906.txt
-\tan (5 t+7)
t -tan(5*t + 7)
8094.txt
\sin \left(\sqrt[3]{-\frac{19 t}{5}-\frac{27}{5}}\right)
sin(1.75441064292772*(-0.703703703703704*t - 1)**0.333333333333333)
t sin(1.75441064292772*(-0.703703703703704*t - 1)**0.333333333333333)
12803.txt
\sqrt[3]{4-6 t}
t (4 - 6*t)**0.333333333333333
43478.txt
\frac{1}{\left(t^3+1\right)^5}
(t**3 + 1)**(-5)
t (t**3 + 1)**(-5)
39368.txt
e^{\cos \left(8 t+\frac{13}{5}\right)}
t exp(cos(8*t + 2.6))
14932.txt
\tan \left(6 t^5+7\right) (-\csc (8 t))
-tan(6*t**5 + 7)*csc(8*t)
t -tan(6*t**5 + 7)*csc(8*t)
216.txt
\sqrt[3]{-\sin (6)}
(-sin(6))**(1/3)
None None
15360.txt
\sqrt[3]{6 t-5}
t (6*t - 5)**(1/3)
13156.txt
\sqrt{2 t-9}
t sqrt(2*t - 9)
46335.txt
\cos \left(\sqrt{5-2 t^5}\right)
cos(sqrt(5 - 2*t**5))
t cos(sqrt(5 - 2*t**5))
37078.txt
\sqrt[3]{-8 t-6}+\sqrt[3]{-4 t-6}
t (-8*t - 6)**(1/3) + (-4*t - 6)**(1/3)
29547.txt
-\tan ^{-1}(1-9 t)
atan(9*t - 1)
t atan(9*t - 1)
29469.txt
e^{7-2 t}-\sin (2-t)
t exp(7 -